<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [1]:
import pandas as pd
import tweepy

# Leer las llaves
keys = pd.read_csv("mis_llaves.csv", sep=';',header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [2]:
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)
tweets = tweepy.Cursor(api.search,
                       q=['paro nacional -RT'],
                       lang='es',
                       geocode=f'4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False).items(100)

In [3]:
t = [tweet for tweet in tweets]
t_processed = []

for tweet in t:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogotá")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t_processed.append(temp)

In [4]:
data = pd.DataFrame(t_processed)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data.head()

,tweet,date,city,user_location,id,link
0,@Frankia25Elena @PoliciaColombia Pareciera que...,2021-05-20 21:21:48,Bogotá,"Bogotá, D.C., Colombia",1395489962490015745,https://twitter.com/i/web/status/1395489962490...
1,#noticias Comité Nacional del Paro convoca mar...,2021-05-20 21:19:46,Bogotá,Bogotá,1395489449967067137,https://twitter.com/i/web/status/1395489449967...
2,¡Ganamos! Después de 20 días en pie de lucha c...,2021-05-20 21:19:44,Bogotá,Villavicencio Meta,1395489440299196416,https://twitter.com/i/web/status/1395489440299...
3,@RevistaSemana ¿Qué tan común es que una polic...,2021-05-20 21:16:27,Bogotá,"Bogotá, Colombia",1395488616843972612,https://twitter.com/i/web/status/1395488616843...
4,@JuanaLaRola VIVA EL PARO NACIONAL!,2021-05-20 21:16:20,Bogotá,Bogotá,1395488586678607881,https://twitter.com/i/web/status/1395488586678...


In [5]:
import re

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweet']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
             
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag,"")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")
        
    return text, links, hashtags, mentions

In [6]:
data[['clean', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,city,user_location,id,link,clean,links,hashtags,mentions
0,@Frankia25Elena @PoliciaColombia Pareciera que...,2021-05-20 21:21:48,Bogotá,"Bogotá, D.C., Colombia",1395489962490015745,https://twitter.com/i/web/status/1395489962490...,Pareciera que por estas cosas es que y otr...,[],[],"[@Frankia25Elena, @PoliciaColombia, @IvanCeped..."
1,#noticias Comité Nacional del Paro convoca mar...,2021-05-20 21:19:46,Bogotá,Bogotá,1395489449967067137,https://twitter.com/i/web/status/1395489449967...,Comité Nacional del Paro convoca marchas para...,[],"[#noticias, #Colombia.#Opinion, #PalacioNacion...",[]
2,¡Ganamos! Después de 20 días en pie de lucha c...,2021-05-20 21:19:44,Bogotá,Villavicencio Meta,1395489440299196416,https://twitter.com/i/web/status/1395489440299...,¡Ganamos! Después de 20 días en pie de lucha c...,[https://t.co/hFMxmFeIvp],[],"[@PartidoVerdeCoL, @CamiloRomero, @AntonioSang..."
3,@RevistaSemana ¿Qué tan común es que una polic...,2021-05-20 21:16:27,Bogotá,"Bogotá, Colombia",1395488616843972612,https://twitter.com/i/web/status/1395488616843...,¿Qué tan común es que una policía se quede so...,[],[],[@RevistaSemana]
4,@JuanaLaRola VIVA EL PARO NACIONAL!,2021-05-20 21:16:20,Bogotá,Bogotá,1395488586678607881,https://twitter.com/i/web/status/1395488586678...,VIVA EL PARO NACIONAL!,[],[],[@JuanaLaRola]


In [7]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list

data['emojis'] = data['clean'].apply(lambda text: get_emojis(text))

In [8]:
data.head()

,tweet,date,city,user_location,id,link,clean,links,hashtags,mentions,emojis
0,@Frankia25Elena @PoliciaColombia Pareciera que...,2021-05-20 21:21:48,Bogotá,"Bogotá, D.C., Colombia",1395489962490015745,https://twitter.com/i/web/status/1395489962490...,Pareciera que por estas cosas es que y otr...,[],[],"[@Frankia25Elena, @PoliciaColombia, @IvanCeped...",[]
1,#noticias Comité Nacional del Paro convoca mar...,2021-05-20 21:19:46,Bogotá,Bogotá,1395489449967067137,https://twitter.com/i/web/status/1395489449967...,Comité Nacional del Paro convoca marchas para...,[],"[#noticias, #Colombia.#Opinion, #PalacioNacion...",[],[]
2,¡Ganamos! Después de 20 días en pie de lucha c...,2021-05-20 21:19:44,Bogotá,Villavicencio Meta,1395489440299196416,https://twitter.com/i/web/status/1395489440299...,¡Ganamos! Después de 20 días en pie de lucha c...,[https://t.co/hFMxmFeIvp],[],"[@PartidoVerdeCoL, @CamiloRomero, @AntonioSang...",[]
3,@RevistaSemana ¿Qué tan común es que una polic...,2021-05-20 21:16:27,Bogotá,"Bogotá, Colombia",1395488616843972612,https://twitter.com/i/web/status/1395488616843...,¿Qué tan común es que una policía se quede so...,[],[],[@RevistaSemana],[]
4,@JuanaLaRola VIVA EL PARO NACIONAL!,2021-05-20 21:16:20,Bogotá,Bogotá,1395488586678607881,https://twitter.com/i/web/status/1395488586678...,VIVA EL PARO NACIONAL!,[],[],[@JuanaLaRola],[]


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [9]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [13]:
data['fecha'] = data.date.dt.floor('T').dt.time

temp = pd.DataFrame(data.fecha.value_counts()).reset_index()
temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')

In [15]:
trace = go.Scatter(x=temp.fecha.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Días',
                   line={'color':'#1DA1F2'})

layout = go.Layout(title='Número de tuits por Hora')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [16]:
from collections import Counter

temp_emojis = pd.DataFrame(list(zip(Counter(sum(data.emojis.values,[])).keys(),Counter(sum(data.emojis.values,[])).values())))
temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)
temp_emojis.head()

,emoji,cnt
5,🙃,4
14,💪,3
12,⬇️,2
13,🔥,2
0,🤷🏻,1


In [17]:
trace = go.Bar(x=temp_emojis.emoji.values[:15],
               y=temp_emojis.cnt.values[:15])

layout = go.Layout(title='Emojis más usados')

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)